In [1]:
import pandas as pd

# 1. Load the combined dataset before cleaning
df = pd.read_csv('C:/Users/PcVIP/Desktop/Bootcamp/Week7/Weekly_project_ml/video_game_sales/data/clean/combined_before_title_cleaning.csv')

# 2. Normalize column names to lowercase
df.columns = df.columns.str.lower()

# 3. Normalize the 'name' column
df['name'] = (
    df['name']
    .str.lower()
    .str.strip()
    .str.replace(r"[^a-z0-9 ]", "", regex=True)   # remove special characters
    .str.replace(r"\s+", " ", regex=True)         # normalize whitespace
)

# 4. Count how many games appear on multiple platforms
name_platform_counts = df.groupby('name')['platform'].nunique().sort_values(ascending=False)
multi_platform_titles = name_platform_counts[name_platform_counts > 1]

print(f"Multiplatform titles: {len(multi_platform_titles)}")

# 5. Convert year_of_release to integer (only after dropping nulls)
df = df[df['year_of_release'].notna()]
df['year_of_release'] = df['year_of_release'].astype(int)

# 6. Drop duplicates (same game, platform, and release year)
df_dedup = df.drop_duplicates(
    subset=['name', 'platform', 'year_of_release'],
    keep='first'
).copy()

print(f"Dataset after eliminating duplicates: {df_dedup.shape}")

# 7. Sanity check
assert df_dedup.duplicated(subset=['name', 'platform', 'year_of_release']).sum() == 0


Multiplatform titles: 9954
Dataset after eliminating duplicates: (59380, 11)


In [2]:
import sys
print(sys.executable)


C:\Users\PcVIP\anaconda3\envs\new_base\python.exe


In [3]:
# Null check
print("Missing values:\n", df_dedup.isnull().sum())

# Duplicate check
dup_check = df_dedup.duplicated(subset=['name', 'platform', 'year_of_release'])
print(f"\nDuplicates: {dup_check.sum()}")


Missing values:
 name                   1
platform               0
year_of_release        0
genre                  1
publisher              0
na_sales               0
eu_sales               0
jp_sales               0
other_sales            0
global_sales           0
rating             42932
dtype: int64

Duplicates: 0


In [4]:
df['genre'] = df['genre'].replace({
    'platform': 'platformer',
    'role-playing': 'rpg'
})

rare_genres = ['party', 'mmo', 'board game', 'education', 'sandbox']
df['genre'] = df['genre'].replace(rare_genres, 'other')

In [5]:
rating_map = {
    'E': 'Children',
    'EC': 'Children',
    'E10+': 'Pre-teens',
    'T': 'Teens',
    'M': 'Adults',
    'AO': 'Adults',
    'RP': 'Unrated',
    'K-A': 'Children',
    'Unrated': 'Unrated'
}


In [6]:
df['rating'] = df['rating'].map(rating_map).fillna('Unrated')


In [7]:
df_dedup['rating'] = df_dedup['rating'].map(rating_map).fillna('Unrated')

In [8]:
df_dedup.to_csv('C:/Users/PcVIP/Desktop/Bootcamp/Week7/Weekly_project_ml/video_game_sales/data/clean/video_game_sales_final_cleaned.csv', index=False)


In [9]:
df.columns

Index(['name', 'platform', 'year_of_release', 'genre', 'publisher', 'na_sales',
       'eu_sales', 'jp_sales', 'other_sales', 'global_sales', 'rating'],
      dtype='object')